# Environment Setup

In [2]:
import nest_asyncio
from dotenv import find_dotenv, load_dotenv
from pprint import pprint

nest_asyncio.apply()
load_dotenv(find_dotenv())


True

# Messages and Chat History

## Data Schemas

- Agent run result `all_messages() -> list[ModelMessage]`
- `ModelMessage` is an `typing.Annotated` of `ModelRequest` and `ModelResponse`
- `ModelRequest` schema:
> ```json
>{
>   parts: "list[ModelRequestPart]",
>   instructions: "str | None",
>   kind: "Literal['request']",
>}
>```

- `ModelResponse` schema:
> ```json
>{
>   parts: "list[ModelResponsePart]",
>   model_name: "str | None",
>   timestamp: "datetime",
>   kind: "Literal['response']",
>}
>```



- parts can be `SystemPromptPart`, `UserPromptPart`, `TextPart`, `ToolCallPart`...
- `SystemPromptPart` schema:
>```JSON
>{
>   content: "str",
>   timestamp: "datetime",
>   dynamic_ref: "str",
>   part_kind: "Literal['system-prompt']"
>}
>```
- `UserPromptPart` schema:
>```JSON
>{
>   content: "str | Sequence[UserContent]",
>   timestamp: "datetime",
>   part_kind: "Literal['user-prompt']",
>}
>```
- `TextPart` schema:
>```JSON
>{
>   content: "str",
>   timestamp: "datetime",
>   part_kind: "Literal['text']",
>}
>```

## Accessing Messages from Results

In [3]:
from pydantic_ai import Agent

agent = Agent('google-gla:gemini-1.5-flash', system_prompt='Be a helpful assistant.')

result = agent.run_sync('Tell me a joke.')

pprint(type(result.all_messages()))


<class 'list'>


In [4]:
print(result.output)

Why don't scientists trust atoms? 

Because they make up everything!



In [5]:
# all messages from the run
pprint(result.all_messages(), indent=2)


[ ModelRequest(parts=[ SystemPromptPart(content='Be a helpful assistant.',
                                        timestamp=datetime.datetime(2025, 4, 28, 6, 32, 41, 662047, tzinfo=datetime.timezone.utc),
                                        dynamic_ref=None,
                                        part_kind='system-prompt'),
                       UserPromptPart(content='Tell me a joke.',
                                      timestamp=datetime.datetime(2025, 4, 28, 6, 32, 41, 662047, tzinfo=datetime.timezone.utc),
                                      part_kind='user-prompt')],
               instructions=None,
               kind='request'),
  ModelResponse(parts=[ TextPart(content="Why don't scientists trust atoms? \n"
                                         '\n'
                                         'Because they make up everything!\n',
                                 part_kind='text')],
                model_name='gemini-1.5-flash',
                timestamp=datetime.dat

## Accessing Messages from Streamed Results

In [6]:
from pydantic_ai.messages import ModelMessage

async def streamed_results() -> list[ModelMessage]:
    async with agent.run_stream("Tell me a joke.") as result:
        # incomplete messages before the stream finishes
        pprint(result.all_messages())

        async for text in result.stream_text():
            print(text)

        # complete messages once the stream finishes
        return result.all_messages()


In [7]:
import asyncio

all_messages = asyncio.run(streamed_results())


[ModelRequest(parts=[SystemPromptPart(content='Be a helpful assistant.',
                                      timestamp=datetime.datetime(2025, 4, 28, 6, 33, 6, 554441, tzinfo=datetime.timezone.utc),
                                      dynamic_ref=None,
                                      part_kind='system-prompt'),
                     UserPromptPart(content='Tell me a joke.',
                                    timestamp=datetime.datetime(2025, 4, 28, 6, 33, 6, 554441, tzinfo=datetime.timezone.utc),
                                    part_kind='user-prompt')],
              instructions=None,
              kind='request')]
Why don't scientists trust atoms? 

Because they make up everything!



In [8]:
pprint(all_messages)

[ModelRequest(parts=[SystemPromptPart(content='Be a helpful assistant.',
                                      timestamp=datetime.datetime(2025, 4, 28, 6, 33, 6, 554441, tzinfo=datetime.timezone.utc),
                                      dynamic_ref=None,
                                      part_kind='system-prompt'),
                     UserPromptPart(content='Tell me a joke.',
                                    timestamp=datetime.datetime(2025, 4, 28, 6, 33, 6, 554441, tzinfo=datetime.timezone.utc),
                                    part_kind='user-prompt')],
              instructions=None,
              kind='request'),
 ModelResponse(parts=[TextPart(content="Why don't scientists trust atoms? \n"
                                       '\n'
                                       'Because they make up everything!\n',
                               part_kind='text')],
               model_name='gemini-1.5-flash',
               timestamp=datetime.datetime(2025, 4, 28, 6, 33, 8,

## Using Messages as Input for Further Agent Runs

The primary use of message histories in PydanticAI is to maintain context across multiple agent runs.

To use existing messages in a run, pass them to the `message_history` parameter of `Agent.run`, `Agent.run_sync` or `Agent.run_stream`.

If `message_history` is set and not empty, a new system prompt is not generated — we assume the existing message history includes a system prompt.

In [15]:
result1 = agent.run_sync("Tell me a joke.")
print(result1.output)


Why don't scientists trust atoms? 

Because they make up everything!



In [16]:
result2 = agent.run_sync("Explain?", message_history=result1.new_messages())
print(result2.output)


The joke plays on the double meaning of "make up."  

* **Literal meaning:** Atoms are the fundamental building blocks of matter, so they literally *make up* everything in the universe.

* **Figurative meaning:**  "Making something up" means to invent or fabricate a lie.

The humor comes from the unexpected shift between these two meanings.  The punchline suggests that because atoms are the basis of reality, they are untrustworthy because they could be "making up" anything.  It's a silly, absurd thought.



In [21]:
pprint(result2.all_messages(), compact=True, indent=2, depth=3)

[ ModelRequest(parts=[ SystemPromptPart(content='Be a helpful assistant.',
                                        timestamp=datetime.datetime(2025, 4, 24, 13, 56, 34, 329847, tzinfo=datetime.timezone.utc),
                                        dynamic_ref=None,
                                        part_kind='system-prompt'),
                       UserPromptPart(content='Tell me a joke.',
                                      timestamp=datetime.datetime(2025, 4, 24, 13, 56, 34, 329854, tzinfo=datetime.timezone.utc),
                                      part_kind='user-prompt')],
               instructions=None,
               kind='request'),
  ModelResponse(parts=[ TextPart(content="Why don't scientists trust atoms? \n"
                                         '\n'
                                         'Because they make up everything!\n',
                                 part_kind='text')],
                model_name='gemini-1.5-flash',
                timestamp=datetime.d

## Storing and Loading Messages (to JSON)

- When there's a need to store the messages history of an agent run on disk or in a database. This might be for evals, for sharing data between Python and JavaScript/TypeScript, or any number of other use cases.
- PydanticAI uses a `TypeAdapter` to provide this capability. You can use `ModelMessagesTypeAdapter` or create a custom one.

In [22]:
from pydantic_core import to_jsonable_python
from pydantic_ai.messages import ModelMessagesTypeAdapter

result1 = agent.run_sync("Tell me a joke.")
history_step_1 = result1.all_messages()

pprint(history_step_1)

[ModelRequest(parts=[SystemPromptPart(content='Be a helpful assistant.',
                                      timestamp=datetime.datetime(2025, 4, 24, 14, 2, 49, 740980, tzinfo=datetime.timezone.utc),
                                      dynamic_ref=None,
                                      part_kind='system-prompt'),
                     UserPromptPart(content='Tell me a joke.',
                                    timestamp=datetime.datetime(2025, 4, 24, 14, 2, 49, 740984, tzinfo=datetime.timezone.utc),
                                    part_kind='user-prompt')],
              instructions=None,
              kind='request'),
 ModelResponse(parts=[TextPart(content="Why don't scientists trust atoms? \n"
                                       '\n'
                                       'Because they make up everything!\n',
                               part_kind='text')],
               model_name='gemini-1.5-flash',
               timestamp=datetime.datetime(2025, 4, 24, 14, 2, 

In [23]:
as_python_objects = to_jsonable_python(history_step_1)

pprint(as_python_objects)

[{'instructions': None,
  'kind': 'request',
  'parts': [{'content': 'Be a helpful assistant.',
             'dynamic_ref': None,
             'part_kind': 'system-prompt',
             'timestamp': '2025-04-24T14:02:49.740980Z'},
            {'content': 'Tell me a joke.',
             'part_kind': 'user-prompt',
             'timestamp': '2025-04-24T14:02:49.740984Z'}]},
 {'kind': 'response',
  'model_name': 'gemini-1.5-flash',
  'parts': [{'content': "Why don't scientists trust atoms? \n"
                        '\n'
                        'Because they make up everything!\n',
             'part_kind': 'text'}],
  'timestamp': '2025-04-24T14:02:50.445901Z'}]


In [24]:
same_history_as_step_1 = ModelMessagesTypeAdapter.validate_python(as_python_objects)
pprint(same_history_as_step_1)

[ModelRequest(parts=[SystemPromptPart(content='Be a helpful assistant.',
                                      timestamp=datetime.datetime(2025, 4, 24, 14, 2, 49, 740980, tzinfo=TzInfo(UTC)),
                                      dynamic_ref=None,
                                      part_kind='system-prompt'),
                     UserPromptPart(content='Tell me a joke.',
                                    timestamp=datetime.datetime(2025, 4, 24, 14, 2, 49, 740984, tzinfo=TzInfo(UTC)),
                                    part_kind='user-prompt')],
              instructions=None,
              kind='request'),
 ModelResponse(parts=[TextPart(content="Why don't scientists trust atoms? \n"
                                       '\n'
                                       'Because they make up everything!\n',
                               part_kind='text')],
               model_name='gemini-1.5-flash',
               timestamp=datetime.datetime(2025, 4, 24, 14, 2, 50, 445901, tzinfo=T

In [25]:
result2 = agent.run_sync(
    "Tell me a different joke.", message_history=same_history_as_step_1
)
pprint(result2.all_messages())

[ModelRequest(parts=[SystemPromptPart(content='Be a helpful assistant.',
                                      timestamp=datetime.datetime(2025, 4, 24, 14, 2, 49, 740980, tzinfo=TzInfo(UTC)),
                                      dynamic_ref=None,
                                      part_kind='system-prompt'),
                     UserPromptPart(content='Tell me a joke.',
                                    timestamp=datetime.datetime(2025, 4, 24, 14, 2, 49, 740984, tzinfo=TzInfo(UTC)),
                                    part_kind='user-prompt')],
              instructions=None,
              kind='request'),
 ModelResponse(parts=[TextPart(content="Why don't scientists trust atoms? \n"
                                       '\n'
                                       'Because they make up everything!\n',
                               part_kind='text')],
               model_name='gemini-1.5-flash',
               timestamp=datetime.datetime(2025, 4, 24, 14, 2, 50, 445901, tzinfo=T

## Other Ways of Using Messages

- Since messages are defined by simple dataclasses, you can manually create and manipulate, e.g. for testing.
- The message format is independent of the model used, so you can use messages in different agents, or the same agent with different models.

In [26]:
result1 = agent.run_sync("Tell me a joke.")
print(result1.output)

Why don't scientists trust atoms? 

Because they make up everything!



In [27]:
result2 = agent.run_sync(
    "Explain?",
    model="google-gla:gemini-1.5-pro",
    message_history=result1.new_messages(),
)
print(result2.output)

The joke plays on the double meaning of "make up."  

* **Make up (meaning 1: invent):**  This implies that atoms are lying or not telling the truth.
* **Make up (meaning 2: constitute):**  This refers to the scientific fact that atoms are the fundamental building blocks of all matter.

The humor comes from the unexpected shift between the common understanding of "making things up" (like a story) and the scientific meaning.



In [28]:
pprint(result2.all_messages())

[ModelRequest(parts=[SystemPromptPart(content='Be a helpful assistant.',
                                      timestamp=datetime.datetime(2025, 4, 24, 14, 15, 7, 768026, tzinfo=datetime.timezone.utc),
                                      dynamic_ref=None,
                                      part_kind='system-prompt'),
                     UserPromptPart(content='Tell me a joke.',
                                    timestamp=datetime.datetime(2025, 4, 24, 14, 15, 7, 768032, tzinfo=datetime.timezone.utc),
                                    part_kind='user-prompt')],
              instructions=None,
              kind='request'),
 ModelResponse(parts=[TextPart(content="Why don't scientists trust atoms? \n"
                                       '\n'
                                       'Because they make up everything!\n',
                               part_kind='text')],
               model_name='gemini-1.5-flash',
               timestamp=datetime.datetime(2025, 4, 24, 14, 15,